In [1]:
!echo $SHELL
!echo $PATH
!echo $JOURNAL_ROOT

/bin/zsh
/Users/tobiaspucher/miniconda3/bin:/opt/homebrew/opt/node@22/bin:/Users/tobiaspucher/miniconda3/bin:/Users/tobiaspucher/miniconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/Apple/usr/bin:/Library/TeX/texbin:/Users/tobiaspucher/.local/bin:/Users/tobiaspucher/Library/Application Support/JetBrains/Toolbox/scripts
/Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal


In [2]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.ERROR)
import random
random.seed(0)
import os
os.getenv('JOURNAL_ROOT')

'/Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal'

In [4]:
from src.tradecli import *
from src.utils import *
from src.journal import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

frozenset({'type_2_d', 'h4', 'type_1_d', 'type_1_w', 'type_1_h4', 'type_2_h4', 'type_2_m15', 'w', 'type_1_m15', 'type_2_w', 'm1', 'type_2_m1', 'M', 'type_2_M', 'type_1_m5', 'type_1_m1', 'type_1_m30', 'type_1_h1', 'type_2_h1', 'm30', 'type_1_M', 'h1', 'm15', 'd', 'type_2_m5', 'm5', 'type_2_m30'})


In [10]:
config = Config(
    include_patterns=[r'^type_1_', r'^risk_reward_ratio$', r'^side'],  
    exclude_patterns=[r'^management_strategy$'],         
    #force_values={'side': 'short', 'side': 'long'},
)

full_df = j.to_dataframe(config)
#full_df = full_df[full_df['side'] == 'long']
display(full_df)

,risk_reward_ratio,type_1_d,type_1_w,type_1_h4,type_1_m15,type_1_m5,side,type_1_m1,type_1_m30,type_1_h1,type_1_M
0,2.000000,False,False,False,False,False,long,True,False,False,False
1,0.666667,False,False,False,False,False,long,False,False,False,False
2,10.000000,False,False,False,False,False,long,False,False,False,False
3,10.000000,False,False,False,False,False,long,False,False,False,False
4,3.000000,True,False,False,False,False,long,False,False,False,False
5,3.943182,False,False,False,False,False,long,False,False,False,True
6,3.323529,False,False,True,False,False,long,False,False,False,False
7,7.927273,False,False,False,False,False,long,True,False,False,False
8,8.811321,False,False,False,False,False,long,False,True,False,False
9,2.986486,False,False,True,False,False,long,False,False,False,False


In [11]:
!rm -rf output
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
j.to_markdown(output_dir='output',)

In [12]:
df = j.to_dataframe(config=None)
df

,trade_uid,risk_reward_ratio,type_2_d,sl_price,type_1_d,type_1_w,entry_time,type_1_h4,management_strategy,type_2_h4,type_3_h4,type_2_m15,potential_risk_reward,tp_price,type_1_m15,type_2_w,type_3_d,initial_return,SL_distance,type_2_m1,type_2_M,close_price,initial_risk_reward,type_1_m5,htf_poi_ltf_confirmation,side,type_3_m15,type_1_m1,type_1_m30,type_3_m5,type_3_h1,numerical_confidence,entry_price,type_1_h1,outcome,type_2_h1,type_3_m1,type_1_M,type_3_m30,type_3_w,potential_return,type_3_M,type_2_m5,return,TP_distance,potential_price,session,type_2_m30,htf_poi_ltf_confirmation_False,htf_poi_ltf_confirmation_True,management_strategy_be_after_1r,management_strategy_be_after_push,management_strategy_no_management,management_strategy_strategy_2,outcome_loss,outcome_win
0,1,2.000000,False,1.0950,False,False,2025-01-10 20:47:38.336306,False,no_management,False,False,False,NaN,1.1100,False,False,False,0.0100,0.0050,False,False,NaN,2.000000,False,False,long,True,True,False,False,False,3.0,1.1000,False,NaN,False,False,False,False,False,NaN,False,False,NaN,0.0100,NaN,tokyo,False,True,False,False,False,True,False,False,False
1,2,0.666667,False,1.1850,False,False,2025-01-10 20:47:38.336333,False,no_management,False,False,False,NaN,1.2100,False,False,False,0.0100,0.0150,False,False,1.1855,0.666667,False,True,long,False,False,False,False,False,2.0,1.2000,False,loss,True,False,False,False,False,NaN,False,False,-0.0145,0.0100,NaN,tokyo,False,False,True,False,False,True,False,True,False
2,3,10.000000,False,1.2970,False,False,2024-12-27 10:00:01.000000,False,no_management,False,False,False,NaN,1.3300,False,False,False,0.0300,0.5000,False,False,NaN,10.000000,False,False,long,False,False,False,False,False,NaN,1.3000,False,NaN,False,False,False,False,False,NaN,False,False,NaN,0.0300,NaN,london,False,True,False,False,False,True,False,False,False
3,4,10.000000,False,1.2970,False,False,2024-12-25 10:00:00.000000,False,strategy_2,False,False,False,5.000000,1.3300,False,False,False,0.0300,0.5000,False,False,1.3050,10.000000,False,False,long,False,False,False,False,False,4.0,1.3000,False,win,False,False,False,False,False,0.0150,False,False,0.0050,0.0300,1.3150,london,False,True,False,False,False,False,True,False,True
4,5,3.000000,True,1.1018,True,False,2025-01-10 20:47:38.336956,False,be_after_1r,False,False,False,3.397959,1.1410,False,False,False,0.0294,0.0098,False,False,1.1248,3.000000,False,False,long,False,False,False,False,False,3.0,1.1116,False,win,False,False,False,False,False,0.0333,False,False,0.0132,0.0294,1.1449,tokyo,False,True,False,True,False,False,False,False,True
5,6,3.943182,False,1.1125,False,False,2025-01-10 20:47:38.336986,False,no_management,False,False,True,1.590909,1.1560,False,False,False,0.0347,0.0088,False,False,NaN,3.943182,False,True,long,False,False,False,False,False,3.0,1.1213,False,NaN,False,False,True,False,False,0.0140,False,False,NaN,0.0347,1.1353,tokyo,False,False,True,False,False,True,False,False,False
6,7,3.323529,False,1.1256,False,False,2025-01-10 20:47:38.337009,True,be_after_push,False,False,False,6.558824,1.1550,False,True,False,0.0226,0.0068,False,False,1.1206,3.323529,False,False,long,False,False,False,False,False,4.0,1.1324,False,loss,False,False,False,False,False,0.0446,False,False,-0.0118,0.0226,1.1770,tokyo,False,True,False,False,True,False,False,True,False
7,8,7.927273,False,1.1411,False,False,2025-01-10 20:47:38.337029,False,be_after_push,False,False,False,5.400000,1.1902,False,False,False,0.0436,0.0055,False,True,1.1580,7.927273,False,True,long,False,True,False,False,False,5.0,1.1466,False,win,False,False,False,False,False,0.0297,False,False,0.0114,0.0436,1.1763,tokyo,False,False,True,False,True,False,False,False,True
8,9,8.811321,False,1.1328,False,False,2025-01-10 20:47:38.337047,False,no_management,True,False,False,5.264151,1.1848,False,False,False,0.0467,0.0053,False,False,NaN,8.811321,False,True,long,False,False,True,False,False,5.0,1.1381,False,NaN,False,False,False,False,False,0.0279,False,False,N

In [13]:
df_test = df.copy()
features = ['type_1_m5', 'type_2_m5', 'type_3_m5', 'htf_poi_ltf_confirmation', 'return']
df_test = df_test[features]
df_test = df_test.dropna()
exptected_return = df_test['return'].mean()
print('Expected return full search space:', exptected_return)
if exptected_return > 0:
    print('Expected return is positive')
else:
    print('Expected return is negative')
df_test

display(df_test.dtypes)
display(df_test.describe())
display(df_test)


report_best_performing_tags_value(df_test, y_col='return', score_mode='mean', method='brute_force')

Expected return full search space: 0.0031624999999999848
Expected return is positive


type_1_m5                       bool
type_2_m5                       bool
type_3_m5                       bool
htf_poi_ltf_confirmation    category
return                       float64
dtype: object

,return
count,8.000000
mean,0.003162
std,0.012738
min,-0.014500
25%,-0.009250
50%,0.008200
75%,0.013325
max,0.016700


,type_1_m5,type_2_m5,type_3_m5,htf_poi_ltf_confirmation,return
1,False,False,False,True,-0.0145
3,False,False,False,False,0.0050
4,False,False,False,False,0.0132
6,False,False,False,False,-0.0118
7,False,False,False,True,0.0114
10,False,False,False,True,-0.0084
11,False,True,False,False,0.0137
12,False,False,False,False,0.0167


Best subset: ('type_1_m5',)
Best combination: (False,)
Best score: 0.0031624999999999848
Best subset: ('type_2_m5',)
Best combination: (True,)
Best score: 0.013700000000000045
Best subset: ('type_2_m5',)
Best combination: (True,)
Best score: 0.013700000000000045


In [14]:
import src.tradecli
cfg = src.tradecli.ReportOptunaConfig(study_name='default_1')
res = report_best_performing_tags_value(df_test, 
                                  y_col='return', 
                                  score_mode='mean', method='optuna', config=cfg)

[I 2025-01-10 20:50:44,441] A new study created in memory with name: default_1


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-01-10 20:50:44,458] Trial 0 finished with value: -0.0038333333333333552 and parameters: {'include_type_1_m5': True, 'value_type_1_m5': False, 'include_type_2_m5': False, 'include_type_3_m5': True, 'value_type_3_m5': False, 'include_htf_poi_ltf_confirmation': True, 'value_htf_poi_ltf_confirmation': True}. Best is trial 0 with value: -0.0038333333333333552.
[I 2025-01-10 20:50:44,462] Trial 1 finished with value: 0.0016571428571428332 and parameters: {'include_type_1_m5': True, 'value_type_1_m5': False, 'include_type_2_m5': True, 'value_type_2_m5': False, 'include_type_3_m5': True, 'value_type_3_m5': False, 'include_htf_poi_ltf_confirmation': False}. Best is trial 1 with value: 0.0016571428571428332.
[I 2025-01-10 20:50:44,467] Trial 2 finished with value: 0.0031624999999999848 and parameters: {'include_type_1_m5': True, 'value_type_1_m5': False, 'include_type_2_m5': False, 'include_type_3_m5': True, 'value_type_3_m5': False, 'include_htf_poi_ltf_confirmation': False}. Best is tr

In [15]:
res

(<optuna.study.study.Study at 0x154198f20>,
 {'include_type_1_m5': True,
  'value_type_1_m5': False,
  'include_type_2_m5': True,
  'value_type_2_m5': True,
  'include_type_3_m5': True,
  'value_type_3_m5': False,
  'include_htf_poi_ltf_confirmation': True,
  'value_htf_poi_ltf_confirmation': False},
 0.013700000000000045)

In [16]:
res[0].best_trial
study = res[0]

In [17]:
# visualize stuff
import optuna
import plotly
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_slice
from optuna.visualization import plot_param_importances

# disable optuna warnings
optuna.logging.set_verbosity(optuna.logging.ERROR)

plot_optimization_history(study).show()
plot_param_importances(study).show()
plot_slice(study).show()

In [38]:
!rm -rf output/studies.md
import pprint
def markdown_report_study(study:optuna.Study, output_dir='output'):
    SECTION_OPTUNA_STUDIES = '## Optuna Studies'
    SECTION_BEST_TRIAL = f'### {study.study_name} - {study.best_trial.number}'
    GENERATION_TIME = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    best_trial = study.best_trial
    best_trial_params = best_trial.params
    best_trial_value = best_trial.value
    best_trial_number = best_trial.number
    best_trial_number
    best_trial_params
    best_trial_value
    # add the params as nicely formatted json inside code block
    best_trial_params_str = f'''
```json
{pprint.pformat(best_trial_params)}
```
''' 
    
    best_trial_str = f'''
{SECTION_BEST_TRIAL}
- generation time: {GENERATION_TIME}
- best trial value: {best_trial_value}
- best trial params: {best_trial_params_str}
'''
    with open(f'{output_dir}/studies.md', 'a') as f:
        f.write(best_trial_str)

    
    # find the section or create it and add md link to the best trial in the study
    with open(f'{output_dir}/index.md', 'r') as f:
        lines = f.readlines()
        found = False
        for i, line in enumerate(lines):
            if SECTION_OPTUNA_STUDIES in line:
                found = True
                break
        if not found:
            lines.append(f'\n{SECTION_OPTUNA_STUDIES}\n')
        lines.append(f'- [[studies.md{SECTION_BEST_TRIAL}]]\n')
    with open(f'{output_dir}/index.md', 'w') as f:
        f.write(''.join(lines))
        

markdown_report_study(study, output_dir='output')